In [1]:
%matplotlib inline
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from math import sqrt
import pandas as pd
import numpy as np
import os, random, sys

# join result path
JoinResPath = "./data/JoinResults/"
# list of join result file names
Ilist = [i for i in os.listdir(JoinResPath) if not i.startswith(".")]

# read join result, if id is not given, randomly read one, 
# least only works when id is not given, i.e., if returned join result
# does not contain enough points, it will recusively call itself again
def readJR(Id):
    Iid = (str(Id) if type(Id)==int else Id) if Id else random.choice(Ilist)
    with open(JoinResPath+Iid,"rb") as f:
        res = []
        for i in f:
            res.append(i.strip().split(","))
    df = pd.DataFrame(res[1:])
    df[1],df[2],df[3] = pd.to_datetime(df[1]), df[2].astype(float), df[3].astype(float) 
    return (int(res[0][0]),float(res[0][1]),float(res[0][2])),df.sort([0,1])# sort_value doesn't work on server's version

# plot scatter of join result
# c is center of intersection, which will be ploted in red dot
# df is the trajectory points in the form of pandas dataframe
# kargs is the key parameters, that will be passed to the plt.scatter func
# inorder to customize the scatter plot params in global scope
def plotJR(c,df,**kargs):
    plt.scatter(df[2],df[3],**kargs)
    plt.scatter(float(c[1]),float(c[2]),color="r",s=30)
    xl,xu,yl,yu=df[2].min(),df[2].max(),df[3].min(),df[3].max()
    xrng,yrng = xu-xl or 0.001,yu-yl or 0.001
    plt.xlim([xl-.1*xrng,xu+.1*xrng])
    plt.ylim([yl-.1*yrng,yu+.1*yrng])
    plt.title("Intersection:%d, # of points:%d"%(c[0],len(df)))

# given a trajectories data frame, split it according to the time interval.
# the rng is the params passed to the dateOffset, which will define the interval.
# if two adjacent trajectory points' time intervel is grearter than this DateOffset,
# it will be divided into to two trajectories
# return result will be a list of dataframe, ordered by their first point's timestamp
def spliTraj(df,**rng):
    curr, last = None, None
    currID, lastID = None, None
    split=[0]
    for i in xrange(df.shape[0]):
        if type(curr)!=type(None):
            last, lastID = curr,currID
        curr,currID = df.iloc[i,1],df.iloc[i,0]
        if type(last)!=type(None):
            if curr>pd.DateOffset(**rng)+last or currID!=lastID:
                split.append(i)
    split.append(i+1)
    #return split
    ret = []
    for i in xrange(1,len(split)):
        splited=df.iloc[split[i-1]:split[i],:]
        ret.append(splited)
    return sorted(ret,cmp=lambda x,y:cmp(x.iloc[0,1],y.iloc[0,1]))
    
# Just like plotJR, but it will plot in the form of line.
# this func takes the results from spliTraj, so dfs is the list of dataframe
# c is still the center of the intersection, and kargs is for customizing the plot
# params in global scope
def plotJR_line(c,dfs,**kargs):
    plt.scatter(c[1],c[2],color="r",s=30)
    if type(dfs)!=list:dfs=[dfs]
    for df in dfs: 
        for t in set(df.iloc[:,0]):    
            if len(df)>1:plt.plot(df.iloc[:,2],df.iloc[:,3],**kargs)
    dfs=pd.concat(dfs)
    xl,xu,yl,yu=dfs[2].min(),dfs[2].max(),dfs[3].min(),dfs[3].max() 
    xrng,yrng = xu-xl or 0.001,yu-yl or 0.001
    plt.xlim([xl-.1*xrng,xu+.1*xrng])
    plt.ylim([yl-.1*yrng,yu+.1*yrng])
    plt.title("Intersection:%d, # of points:%d"%(c[0],len(dfs)))

# given a range and start time, filter the df (range qurey).
# no rng return df itself, 
# df could be a list of df, that actually return from splitTraj,
# or it is just a df, not list. No matter what, func will just
# filter the row of the dataframe by time range query mask
def timeRange(df,time,**rng):
    if len(rng)==0:return df
    if type(df)!=list: 
        mask = df.ix[:,1]<time+pd.DateOffset(**rng)
        mask &= df.ix[:,1]>=time
        return None if df.ix[mask,:].empty else df.ix[mask,:]
    else:
        ret = []
        for i in df:
            tmp=timeRange(i,time,**rng)
            if type(tmp)!=type(None):ret.append(tmp)
        return ret

# given the result from splitTraj, extract the info of intervals, like speed,
# directions, the direction will be normalized into unit vector
# the interval info will be called features, to contribute the inferring 
# verbose tells the progress, but makes cell messy
def calc_features(dfs,verbose=0,c=None):
    features=[]
    count=0
    for df in dfs:
        if len(df)<2:continue
        # interval infos.
        tdiff,xdiff,ydiff = [df[i].diff().iloc[1:] for i in (1,2,3)]
        # distances
        d = list(np.sqrt(xdiff**2+ydiff**2))
        # direction vectors
        xd = [0 if np.isnan(i) else i for i in (np.divide(xdiff,d))]
        yd = [0 if np.isnan(i) else i for i in (np.divide(ydiff,d))]
        # speed
        v = list(d/tdiff.astype('timedelta64[s]'))
        stacked = [v,xd,yd]
        # distances from intersections
        if c:
            x_from_center, y_from_center = np.array(df[2]-c[1]), np.array(df[3]-c[2])
            dist_sq_from_center = x_from_center**2+y_from_center**2
            stacked.append(dist_sq_from_center[1:]+dist_sq_from_center[:-1])
        features.append({"time":list(df[1]),"status":np.vstack(stacked).T})
        if verbose:
            count+=1
            print "\r%d/%d" % (count,len(dfs))
    return features

# tic tac ...
import time
class Timer():
    def __init__(self,msg_last="elasped time:",verbose=1):
        self.msg=""
        self.msg_last=msg_last
        self.verbose=verbose
    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        if self.verbose:
            self.end = time.time()
            self.interval = self.end - self.start
            print self.msg+self.msg_last,"%.1f"%self.interval
            sys.stdout.flush()

In [2]:
import pickle
# def some func to save results
def dump(obj,path):
    with open(path,"wb") as f:
        pickle.dump(obj,f,protocol=pickle.HIGHEST_PROTOCOL)
def load(path):
    with open(path,"rb") as f:
        return pickle.load(f)

In [3]:
SPLIT_INTERVAL = {"minutes":0,"hours":0,"seconds":30}
TIME_RNG = {"hours":24}
#??????????????
DIRECTION_NUM=4# ? how to select it automatically
#??????????????
COS_POW = 2 # set 0 to turn off direction, higher more weighted by one direction
DIST_VAR = 1e-3 # set 1 to make it nearly no difference, less means more weighted by distance
transform_factor = 10

def execute(_id,verbose=0):
    def monitoring(monitor,feature,direction): 
        starts = [(i-pd.Timestamp("00:00:00")).seconds for i in feature["time"]]
        y = feature["status"][:,0]/np.exp(feature["status"][:,3]/DIST_VAR)
        if type(direction) != type(None):
            cosine = np.dot(feature["status"][:,1:3],direction)/sqrt(2)
            similarity = map(lambda x:max([0,x]),cosine**COS_POW)
            y = y*similarity
        for i in xrange(len(y)):
            for j in xrange(starts[i],starts[i+1]):
                monitor[j].append(y[i])

    def wrapped(x):
        if len(x)==0:return 0
        return np.nanmean([i for i in x if np.isfinite(i)])
    # reading
    with Timer("reading:",verbose):
        center,points = readJR(_id)
    # print "number of points:",len(points),"...",
    if len(points)<1000:return None
    # spliting
    with Timer("spliting:",verbose):
        splitTJ = spliTraj(points,**SPLIT_INTERVAL)
    # filtering
    with Timer("filtering:",verbose):
        rngres = timeRange(splitTJ,pd.Timestamp("00:00:00"),**TIME_RNG)
    # calculating interval attrs
    with Timer("features:",verbose):
        features=calc_features(rngres,c=center)
    # directions
    with Timer("dir:",verbose):
        cuml_dirs = []
        for i in features:
            filter_ = i["status"][:,1:3].sum(axis=1)!=0.0
            ##remove 1,0
            filter_ &= abs(i["status"][:,1:3].sum(axis=1))!=1
            cuml_dirs.append(i["status"][filter_,1:3])
        cuml_dirs = np.vstack(cuml_dirs)
        model = KMeans(n_clusters=DIRECTION_NUM)
        model.fit(cuml_dirs)
        directions = model.cluster_centers_   
        labels = model.labels_
    # monitoring
    with Timer("monitoring:",verbose):
        speed_monitors=[]            
        for direction in directions:
            speed_monitor=[[] for i in xrange(86400)]
            for feature in features:
                monitoring(speed_monitor,feature,direction)
            speed_monitors.append(speed_monitor)
        # trends   
        trends=[]
        for i in xrange(len(directions)):
            trend = []
            for j in speed_monitors[i]:
                trend.append(wrapped(j))
            trends.append(1-1/(1+transform_factor*len(points)*np.array(trend)))
#---collecting result---#
    collector={}
    collector["len"]=len(points)
    collector["dirs"]=directions
    collector["features"]=features
    collector["trends"]=trends
    collector["coef"]=np.corrcoef(trends)
#-----------------------# 
    return collector

In [23]:
from multiprocessing import Process
Monitoring_PATH = "./data/MoniteringResults/" 

# set true to run multi processing
MULTIPLE=True
# number of processors
nthread=24
# get unfinished jobs
unfinished = list(set(Ilist)-set(os.listdir(Monitoring_PATH)))
print len(unfinished),"left"

# give each processor the job
# job is executor + part of unfinished list
def job_generator(executor,joblist,thread_id):
    def func():
        for _id in joblist:
            executor(_id,thread_id)
    return func

# given _id(intersection id), run the work plus some prints
def executor(_id,thread_id=0):
    with Timer() as t:
        t.msg+="Executed by thread %d #%s ..."%(thread_id,_id)
        if _id in os.listdir(Monitoring_PATH): 
            t.msg+= " already finished. "
            return
        res = execute(_id)
        if not res: 
            t.msg+=" Too small, jumped. "
            dump(res,Monitoring_PATH+str(_id))
            return
        t.msg+= ("# of points: %d "%res["len"])
        dump(res,Monitoring_PATH+str(_id))
        t.msg+="done. "

# auto-start/auto-terminate class for processors
class p_exit:
    def __init__(self,quene):
        self.quene=quene
    def __enter__(self):
        for q in quene:
            q.start()
    def __exit__(self,*arg):
        for q in quene:
            q.terminate()
        
if MULTIPLE:
    print "Start multiprocessing model"
    avg_length = len(unfinished)/nthread
    quene=[]
    for i in xrange(nthread):
        if i+1==nthread:# last handle rest
            func = job_generator(executor,unfinished[i*avg_length:],i)
        else:
            func = job_generator(executor,unfinished[i*avg_length:(i+1)*avg_length],i)
        p=Process(target=func)
        quene.append(p)
    print "Job assigned"
    with p_exit(quene):
        for q in quene:
            q.join()
else:             
    for _id in sorted(Ilist):
        executor(_id)
    
print "All set!"

0 left
Start multiprocessing model
Job assigned
All set!


In [ ]:
_id = 1117

plt.figure(figsize=(8,5))
plotJR(*readJR(_id),s=1,color="b",alpha=.3)
plt.show()

In [21]:
start,end = 170,500
Monitoring_PATH = "./data/MoniteringResults/" 

for _id in [] or sorted(os.listdir(Monitoring_PATH))[start:end]:
    res=load(Monitoring_PATH+str(_id))
    if res is None:continue
    else: 
        print _id,
        res["len"]

1157 1158 1159 116 1160 1162 1163 1165 1166 1167 1168 1169 1173 1174 1175 1177 1178 1179 118 1180 1181 1182 1185 1186 1187 1188 1189 119 1190 1192 1193 1194 1195 1196 1197 1198 1199 120 1200 1201 1202 1203 1205 1206 121 1211 1212 1213 1214 1215 1216 1217 1219 122 1220 1221 1222 1223 1227 123 1230 1231 1232 1233 1234 1237 1238 1240 1242 1243 1244 1245 1246 1248 125 1250 1251 1252 1254 1255 1256 1258 1260 1261 1262 1263 1265 1267 1268 1269 127 1271 1272 1274 1275 1276 1277 1278 1279 128 1281 1282 1283 1284 1286 1287 1288 1289 129 1290 1291 1292 1293 1294 1296 1297 1298 1299 13 130 1300 1301 1302 1303 1307 1308 1309 131 1310 1311 1312 1313 1314 1316 1317 1319 132 1320 1321 1322 1324 1325 1326 1327 1328 1329 133 1330 1332 1334 1335 1336 1337 1338 1339 134 1341 1342 1343 1344 1345 1346 1347 1348 135 1350 1352 1353 1355 1356 1357 1359 1360 1361 1362 1363 1364 1365 1366 1367 1369 137 1371 1373 1375 1377 1378 1379 138 1380 1382 1384 1385 1386 1388 139 1390 1391 1393 1394 1395 1396 1397 1399 14

In [20]:
!rm data/MoniteringResults/1172
